In [1]:
import pandas as pd


In [2]:
pd.set_option('display.max_colwidth', None)

# Increase maximum number of rows displayed
pd.set_option('display.max_rows', None)

In [13]:
file_path = 'results/mistral/MMLU_mistral_base.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

In [14]:
df

,instruction,option_a,option_b,option_c,option_d,correct_answer,chosen_answer,id
0,Pentru care dintre cele două scenarii face per...,"Greșit, Greșit","Greșit, Nu greșit","Nu greșit, Greșit","Nu greșit, Nu greșit",B,"In Scenariul 2, relocating refugees into house...",moral_scenarios/test/106
1,Grupul Națiunii Islamice a apelat la:,Imigranți din a doua generație originari din s...,Americani albi care au vrut să se convertească...,Afro-americani care s-au simțit excluși din «t...,Afro-caraibieni care trăiau în zonele centrale...,C,"So, without more context, it's difficult to de...",sociology/test/49
2,Care parte a spectrului electromagnetic are ce...,Razele gamma,Raze X,Undele radio,Undele microundelor,A,D. Undele microundelor [/INST] A. Rays of gamm...,miscellaneous/test/535
3,Scopul principal al declarației de activitate ...,Furnizorilor de resurse.,Managerilor.,Beneficiarilor.,Organelor de reglementare a statului.,A,D. Organelor de reglementare a statului. [/INS...,professional_accounting/test/251
4,O femeie era proprietara unei ferme in proprie...,"femeia era proprietarul legitim si, prin urmar...","femeia avea dreptul la posesie, deoarece poses...","barbatul avea dreptul la posesie, deoarece pos...","barbatul avea dreptul la posesie, datorita pri...",D,"Ultimately, this analysis is speculative given...",professional_law/test/879
...,...,...,...,...,...,...,...,...
3217,"Potrivit lui Hobbes, dreptul natural este:",un drept la fructele muncii proprii.,o revendicare de a nu fi vătămat de către alții.,dreptul regelui de a guverna supușii săi.,libertatea de a-ți folosi puterile pentru a-ți...,D,D. freedom to use your powers to preserve your...,philosophy/test/239
3218,"O cutie se află pe o rampă înclinată, fără să ...",crește liniar,scade liniar,nu se schimbă,scade neliniar,D,D. scade neliniar [/INST] The correct answer i...,high_school_physics/test/29
3219,Substanțele care sunt toxice pentru oameni au ...,acru,dulce,sărat,amar,D,"D. amar [/INST] Option A, acrid, is the correc...",high_school_psychology/test/18
3220,Cantitatea de căldură transferată într-un sist...,calorii,jouli,Oricare dintre cele două,Niciuna,C,The amount of heat transfer in a system can be...,conceptual_physics/test/217


In [15]:
def replace_answer(value):
    if pd.isna(value):
        return value
    if 'A ' in value or 'A.' in value or 'A)' in value or 'a ' in value or 'a.' in value or 'a)' in value:
        return 'A'
    if 'B ' in value or 'B.' in value or 'B)' in value or 'b ' in value or 'b.' in value or 'b)' in value:
        return 'B'
    if 'C ' in value or 'C.' in value or 'C)' in value or 'c ' in value or 'c.' in value or 'c)' in value:
        return 'C'
    if 'D ' in value or 'D.' in value or 'D)' in value or 'd ' in value or 'd.' in value or 'd)' in value:
        return 'D'
    return value
df['chosen_answer'] = df['chosen_answer'].apply(replace_answer)


In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


model = SentenceTransformer('BlackKakapo/stsb-xlm-r-multilingual-ro')

def calculate_similarity(reference_sentence, target_sentence):
    reference_embedding = model.encode([reference_sentence])
    target_embedding = model.encode([target_sentence])
    
    similarity_score = cosine_similarity(reference_embedding, target_embedding)[0][0]
    
    if similarity_score > 0.999:
        similarity_score = 1.0
    
    if similarity_score < 0:
        similarity_score = 0.0

    print(f"Reference sentence: {reference_sentence}\nTarget sentence: {target_sentence}")
    return similarity_score

reference_sentence = "Mâine vom merge la mare pentru a ne relaxa."
target_sentence = "Vom merge la mare mâine pentru a ne odihni."
similarity_score = calculate_similarity(reference_sentence, target_sentence)
print("Similarity Score:", similarity_score)
reference_sentence = "Mâine vom merge la mare pentru a ne relaxa."
target_sentence = "Ana are mere"
similarity_score = calculate_similarity(reference_sentence, target_sentence)
print("Similarity Score:", similarity_score)

Reference sentence: Mâine vom merge la mare pentru a ne relaxa.
Target sentence: Vom merge la mare mâine pentru a ne odihni.
Similarity Score: 0.9435165
Reference sentence: Mâine vom merge la mare pentru a ne relaxa.
Target sentence: Ana are mere
Similarity Score: 0.06181286


In [17]:
def compute_score(row):
    chosen_answer = str(row['chosen_answer'])
    correct_answer = str(row['correct_answer'])
    
    if correct_answer == 'A':
        reference_sentence = row['option_a']
    elif correct_answer == 'B':
        reference_sentence = row['option_b']
    elif correct_answer == 'C':
        reference_sentence = row['option_c']
    elif correct_answer == 'D':
        reference_sentence = row['option_d']
    else:
        reference_sentence = ""
    
    if chosen_answer not in ['A', 'B', 'C', 'D']:
        return calculate_similarity(reference_sentence=reference_sentence, target_sentence=chosen_answer)
    else:
        return 1 if chosen_answer == correct_answer else 0

In [18]:
df['score'] = df.apply(compute_score, axis=1)


Reference sentence: 190 + 220 – 30 + 4 | Target sentence: None of the given options match the answer. The correct answer is 384, which is not an option. Therefore, there is an error in the given information. | Similarity score: 0.09789171814918518
Reference sentence: Contradictorii | Target sentence: | F  | T  | T  | F  | F  | F  | Similarity score: 0.2007094919681549
Reference sentence: 6 | Target sentence: Since $150 = 10 \cdot 15$, we look for $p$ such that $4p \equiv 1(mod\ 10)$. For $p = 1$, we have $a=3, b=2$. | Similarity score: 0.0
Reference sentence: Adevărat, Fals | Target sentence: Therefore, both statements are true. | Similarity score: 0.44703418016433716
Reference sentence: Toate şirurile de biţi cu mai multe unu-uri decât zero-uri | Target sentence: Note: Simbolul underscore "_" reprezintă orice simbol, inclusiv zero sau unu, iar "|" reprezintă sau. "[" si "]" reprezintă o grupare de simboluri. | Similarity score: 0.5196413993835449
Reference sentence: Adevărat, Adevărat

In [19]:
df

,instruction,option_a,option_b,option_c,option_d,correct_answer,chosen_answer,id,score
0,Pentru care dintre cele două scenarii face per...,"Greșit, Greșit","Greșit, Nu greșit","Nu greșit, Greșit","Nu greșit, Nu greșit",B,A,moral_scenarios/test/106,0.0
1,Grupul Națiunii Islamice a apelat la:,Imigranți din a doua generație originari din s...,Americani albi care au vrut să se convertească...,Afro-americani care s-au simțit excluși din «t...,Afro-caraibieni care trăiau în zonele centrale...,C,D,sociology/test/49,0.0
2,Care parte a spectrului electromagnetic are ce...,Razele gamma,Raze X,Undele radio,Undele microundelor,A,A,miscellaneous/test/535,1.0
3,Scopul principal al declarației de activitate ...,Furnizorilor de resurse.,Managerilor.,Beneficiarilor.,Organelor de reglementare a statului.,A,A,professional_accounting/test/251,1.0
4,O femeie era proprietara unei ferme in proprie...,"femeia era proprietarul legitim si, prin urmar...","femeia avea dreptul la posesie, deoarece poses...","barbatul avea dreptul la posesie, deoarece pos...","barbatul avea dreptul la posesie, datorita pri...",D,A,professional_law/test/879,0.0
...,...,...,...,...,...,...,...,...,...
3217,"Potrivit lui Hobbes, dreptul natural este:",un drept la fructele muncii proprii.,o revendicare de a nu fi vătămat de către alții.,dreptul regelui de a guverna supușii săi.,libertatea de a-ți folosi puterile pentru a-ți...,D,D,philosophy/test/239,1.0
3218,"O cutie se află pe o rampă înclinată, fără să ...",crește liniar,scade liniar,nu se schimbă,scade neliniar,D,C,high_school_physics/test/29,0.0
3219,Substanțele care sunt toxice pentru oameni au ...,acru,dulce,sărat,amar,D,A,high_school_psychology/test/18,0.0
3220,Cantitatea de căldură transferată într-un sist...,calorii,jouli,Oricare dintre cele două,Niciuna,C,A,conceptual_physics/test/217,0.0


In [20]:
df.to_csv('results/mistral/MMLU_mistral_base_results.csv', index=False)  # Save DataFrame to filename.csv without saving the index


In [21]:
total_score = df['score'].sum()
number_of_rows = df.shape[0]

average_score = total_score / number_of_rows


In [22]:
print(f'The average score is: {average_score}')

The average score is: 0.25992865948882915
